<a href="https://colab.research.google.com/github/gupta-keshav/age_gender_classification/blob/master/alignedAgeSubmit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
%pip install patool
import patoolib
patoolib.extract_archive("/content/drive/My Drive/Adience/datasetAge.rar", outdir="/content")

     |████████████████████████████████| 81kB 3.2MB/s 
patool: Extracting /content/drive/My Drive/Adience/datasetAge.rar ...
patool: running /usr/bin/unrar x -- "/content/drive/My Drive/Adience/datasetAge.rar"
patool:     with cwd='/content'
patool: ... /content/drive/My Drive/Adience/datasetAge.rar extracted to `/content'.


'/content'

In [1]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, AveragePooling2D, PReLU, ReLU
from keras import optimizers, regularizers
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
np.random.seed(48)
from keras.applications import vgg19
from keras.models import Model
input_shape = (256, 256, 3)
vgg = vgg19.VGG19(include_top=False ,weights='imagenet', 
                                     input_shape=input_shape)   

Using TensorFlow backend.


In [0]:
last = vgg.output

In [3]:
x = Flatten()(last)
x = Dropout(0.3)(x)
x = Dense(4096, activation='relu', kernel_regularizer=regularizers.l1_l2(l1=0.1, l2=0.1))(x)
x = Dropout(0.65)(x)


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
preds = Dense(8, activation = 'softmax')(x)

In [0]:
model = Model(vgg.input, preds)

In [6]:
import pandas as pd
layers = [(layer.output, layer.name, layer.trainable) for layer in model.layers]
pd.set_option('max_colwidth', -1)

pd.DataFrame(layers, columns=['Layer Type', 'Layer Name', 'Layer Trainable']) 

,Layer Type,Layer Name,Layer Trainable
0,"Tensor(""input_1:0"", shape=(?, 256, 256, 3), dtype=float32)",input_1,False
1,"Tensor(""block1_conv1/Relu:0"", shape=(?, 256, 256, 64), dtype=float32)",block1_conv1,True
2,"Tensor(""block1_conv2/Relu:0"", shape=(?, 256, 256, 64), dtype=float32)",block1_conv2,True
3,"Tensor(""block1_pool/MaxPool:0"", shape=(?, 128, 128, 64), dtype=float32)",block1_pool,True
4,"Tensor(""block2_conv1/Relu:0"", shape=(?, 128, 128, 128), dtype=float32)",block2_conv1,True
5,"Tensor(""block2_conv2/Relu:0"", shape=(?, 128, 128, 128), dtype=float32)",block2_conv2,True
6,"Tensor(""block2_pool/MaxPool:0"", shape=(?, 64, 64, 128), dtype=float32)",block2_pool,True
7,"Tensor(""block3_conv1/Relu:0"", shape=(?, 64, 64, 256), dtype=float32)",block3_conv1,True
8,"Tensor(""block3_conv2/Relu:0"", shape=(?, 64, 64, 256), dtype=float32)",block3_conv2,True
9,"Tensor(""block3_conv3/Relu:0"", shape=(?, 64, 64, 256), dtype=float32)",block3_conv3,True


In [0]:
for i in range(17):
  model.layers[i].trainable = False

In [8]:
model.compile(optimizer = optimizers.Adam(lr=1e-4), loss= 'categorical_crossentropy',
              metrics = ['accuracy'])

In [9]:
train_datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True
)
train_generator = train_datagen.flow_from_directory(
        '/content/datasetAge/train',
        target_size=(256, 256),
        batch_size=32,
        class_mode='categorical')
validate_generator = train_datagen.flow_from_directory(
        '/content/datasetAge/test',
        target_size=(256, 256),
        batch_size=32,
        class_mode='categorical')



Found 13840 images belonging to 8 classes.
Found 3676 images belonging to 8 classes.


In [10]:
history = model.fit_generator(train_generator, steps_per_epoch=440, epochs=4, verbose=1, callbacks=None, validation_data=validate_generator, validation_steps=110, validation_freq=1, class_weight=None, max_queue_size=10, workers=-1, use_multiprocessing=True, shuffle=False)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/4


/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:716: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:724: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


440/440 [==============================] - 297s 675ms/step - loss: 8839.6398 - acc: 0.3640 - val_loss: 172.7032 - val_acc: 0.4176
Epoch 2/4
440/440 [==============================] - 290s 658ms/step - loss: 172.1778 - acc: 0.4445 - val_loss: 171.5410 - val_acc: 0.4815
Epoch 3/4
440/440 [==============================] - 287s 652ms/step - loss: 171.2112 - acc: 0.5082 - val_loss: 171.7776 - val_acc: 0.4855
Epoch 4/4
440/440 [==============================] - 294s 668ms/step - loss: 170.3744 - acc: 0.5730 - val_loss: 170.0416 - val_acc: 0.5481


In [11]:

train_datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    )
train_generator = train_datagen.flow_from_directory(
        '/content/datasetAge/train',
        target_size=(256, 256),
        batch_size=32,
        class_mode='categorical')
validate_datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    )
validate_generator = validate_datagen.flow_from_directory(
        '/content/datasetAge/test',
        target_size=(256, 256),
        batch_size=32,
        class_mode='categorical')



Found 13840 images belonging to 8 classes.
Found 3676 images belonging to 8 classes.


In [12]:
history = model.fit_generator(train_generator, steps_per_epoch=440, epochs=2, verbose=1, callbacks=None, validation_data=validate_generator, validation_steps=110, validation_freq=1, class_weight=None, max_queue_size=10, workers=-1, use_multiprocessing=True, shuffle=False)

Epoch 1/2


/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:716: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:724: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


440/440 [==============================] - 443s 1s/step - loss: 170.5440 - acc: 0.5215 - val_loss: 169.2725 - val_acc: 0.5560
Epoch 2/2
440/440 [==============================] - 434s 986ms/step - loss: 167.9228 - acc: 0.5546 - val_loss: 166.2843 - val_acc: 0.5720
